This notebook projects a 3D point cloud from lidar onto a camera image.

In [ ]:
import cv2
import numpy as np
import pandas as pd
from cv_bridge import CvBridge
import pcl
import plotly.express as px
import plotly.graph_objects as go
import ros_numpy
import rosbag
from tf.transformations import (
    euler_matrix,
    unit_vector,
    vector_norm,
    quaternion_matrix,
)

# %load_ext blackcellmagic

#### **The following values need to be adjusted for each rosbag**

How to get transformation from lidar to camera frame:
1) Add `static_transform_publisher` with zed and lidar transforms to base_link to a launch file and launch it
2) Run `get_tfs.py` which returns the translation `(x, y, z)` and rotation `(qx, qy, qz, qw)` between two frames
3) Copy them from terminal
4) Insert them below

In [ ]:
# Rosbag path
bag_path = '/home/user/rosbags/repeat/blob/2022-01-20-16-35-51.bag'

# Load rosbag
bag = rosbag.Bag(bag_path)

# ROS topics
camera_topic = '/zed2i/zed_node/left/image_rect_color'
lidar_topic = '/velodyne_points'

# Frame to show (is equivalent to time: frame/10 s)
frame = 100

# Transformation from lidar to camera left optical (using output from tf node)
trans = [0.05999999999999983, -0.3898697311173096, -0.33652837214493014]
quat = [-0.4473902184573541, 0.4473902184573541, -0.5475782979891378, -0.5475782979891378]
# (Euler (rpy)):0.000 -78.500 90.000

Helper functions

In [ ]:
def unpack_bag(bag_path, topic):
    """Extracts all msgs (with time) of a topic from a rosbag"""
    # Load rosbag
    bag = rosbag.Bag(bag_path)

    t_msg = []
    msgs = []
    for topic, msg, t in bag.read_messages(topics=topic):
        msgs.append(msg)
        # Time at which msg was recorded
        t_msg.append(t.to_time())
    return msgs, t_msg


def synchronize_topics(topic1, t_topic1, topic2, t_topic2, t_thresh=0.1):
    """Synchronizes two topics (even if different rate)."""
    if len(topic1) == len(topic2):
        return topic1, topic2

    # Make sure both topics start around the same time and remove first values of
    # signal which is ahead if this is not the case
    topic1, t_topic1, topic2, t_topic2 = correct_for_time_diff(
        topic1, t_topic1, topic2, t_topic2)

    # Assume first array is bigger
    array_big = t_topic1
    array_small = t_topic2
    is_topic1_bigger = True
    # Swap if not
    if len(t_topic1) < len(t_topic2):
        array_small, array_big = array_big, array_small
        is_topic1_bigger = False
    indices = []

    # For each time of the small array, check which is
    # the closest msg in the big array
    for i, v in enumerate(array_small):
        # Offset big array with a msg from small array
        diff = np.abs(array_big - v)
        # Index where time difference is the smallest
        idx = diff.argmin()
        if diff[idx] < t_thresh:
            indices.append(idx)

    print(
        "t_start_diff: {:.3f} s, t_end_diff: {:.3f} s".format(
            np.abs(array_small[0] - array_big[indices][0]),
            np.abs(array_big[indices][-1] - array_small[len(array_big[indices]) - 1]),
        )
    )

    if is_topic1_bigger:
        topic1_synched = topic1[indices]
        topic2_synched = topic2[: len(topic1_synched)]
        return topic1_synched, topic2_synched
    else:
        topic2_synched = topic2[indices]
        topic1_synched = topic1[: len(topic2_synched)]
        return topic1_synched, topic2_synched


def correct_for_time_diff(topic1, t_topic1, topic2, t_topic2, t_thresh=0.1):
    """Checks for a time offset of two topics at the beginning and shorts ahead signal"""
    # Assume first topic is behind and convert to numpy array
    topic_behind, topic_ahead = np.asarray(topic1), np.asarray(topic2)
    t_topic_behind, t_topic_ahead = np.asarray(t_topic1), np.asarray(t_topic2)
    is_topic1_behind = True
    # Swap if other way around
    if t_topic2[0] - t_topic1[0] > t_thresh:
        topic_ahead, topic_behind = topic_behind, topic_ahead
        t_topic_ahead, t_topic_behind = t_topic_behind, t_topic_ahead
        is_topic1_behind = False
    # No offset
    elif np.abs(t_topic1[0] - t_topic2[0]) < t_thresh:
        return tuple([np.asarray(x) for x in [topic1, t_topic1, topic2, t_topic2]])

    # Search for time of ahead topic which is closest to start of behind topic
    diff = np.abs(t_topic_ahead - t_topic_behind[0])
    # Index where time difference is the smallest
    idx = diff.argmin()
    # Remove first couple values to make both start at the same time
    topic_ahead = topic_ahead[idx:]
    t_topic_ahead = t_topic_ahead[idx:]

    if is_topic1_behind:
        print("topic1 was behind by {}".format(idx))
        return topic_behind, t_topic_behind, topic_ahead, t_topic_ahead
    else:
        print("topic2 was behind by {}".format(idx))
        return topic_ahead, t_topic_ahead, topic_behind, t_topic_behind


def cut_pc(pc2D, x1=1280, y1=720, idx=None):
    """Cuts points of 2D pointcloud which lie outside of camera image"""
    if idx is None:
        idx = (
            (pc2D[:, 0] > 0) & (pc2D[:, 0] < x1) & (pc2D[:, 1] > 0) & (pc2D[:, 1] < y1)
        )
        pc_smol = pc2D[idx]
        return pc_smol, idx
    return pc2D[idx]

Load data (camera image and pointcloud) and extract one frame

In [ ]:
# ZED camera image
image, t_image = unpack_bag(bag_path, camera_topic)
# Velodyne point cloud
lidar, t_lidar = unpack_bag(bag_path, lidar_topic)

# Synchronize both topics (zed camera has 30 fps whereas lidar is only 10 Hz)
image, lidar = synchronize_topics(image, t_image, lidar, t_lidar)

# Extract one frame
img = image[frame]
pc = lidar[frame]

# Convert from image msg to numpy array
bridge = CvBridge()
img = bridge.imgmsg_to_cv2(img, desired_encoding="rgb8")
# Convert point cloud msg to numpy array
pc_array = ros_numpy.point_cloud2.pointcloud2_to_xyz_array(pc, remove_nans=True)

How to get camera parameters:
1) Start rosbag with zed recording and echo `camera_info` topic
2) Copy important information
    1) D are the distortion parameters (**needed**)
    2) K is the intrinsic camera matrix (for the raw (distorted) images) (**needed**)
    3) R is the Rectification matrix 
    4) P is the projection/camera matrix
3) Insert them below

In [ ]:
# Lidar points
# Remove points behind car (because not visible in camera frame)
pc_array = pc_array[pc_array[:, 0] > 0]
objectPoints = pc_array

# Get rotation vector from lidar frame to camera frame
# Convert quaternion to rotation matrix
rotMat = quaternion_matrix(quat)[:3, :3]
# Convert rotation matrix to rotation vector
rvec, _ = cv2.Rodrigues(rotMat)

# Translation vector from lidar to camera frame
tvec = np.asarray(trans)

# Camera matrix (K)
cameraMatrix = np.array(
    [
        [519.9877319335938, 0.0, 630.9716796875],
        [0.0, 519.9877319335938, 352.2940673828125],
        [0.0, 0.0, 1.0],
    ]
)

# Distortion coefficients (D)
distCoeffs = np.array([0.0, 0.0, 0.0, 0.0, 0.0])


# Project 3D lidar points onto 2D plane
# Projection of 3D points onto 2D plane
lidar_img_points, _ = cv2.projectPoints(objectPoints, rvec, tvec, cameraMatrix, distCoeffs)
lidar_img_points = lidar_img_points.reshape(len(lidar_img_points), 2)

# Remove points which do not lie inside image plane (add little buffer)
lidar_img_points_small, indices = cut_pc(lidar_img_points, img.shape[1], img.shape[0])
# Same for original pointcloud (used for distance)
pc_array_small = cut_pc(pc_array, idx=indices)

# Create dataframe with x, y coordinates and distance of lidar points
df = pd.DataFrame(lidar_img_points_small, columns=["x", "y"])
# Add distance information
df["dist"] = pc_array_small[:, 0]

### Visualize

In [ ]:
fig = px.imshow(img)
fig.add_trace(
    go.Scatter(
        x=df["x"],
        y=df["y"],
        mode="markers",
        marker=dict(size=3, opacity=0.5, color=df["dist"]),
    )
)
fig.update_layout(width=img.shape[1], height=img.shape[0])
fig.show()

Also project the detected ramp into the image

In [ ]:
# My ROS Node
class VisualDetection():

    def __init__(self):
        pass

    def spin(self, cloud):
        self.cloud = cloud
        # Robosense Lidar has a rate of 10 Hz
        # Convert PointCloud2 msg to numpy array
        pc_array = ros_numpy.point_cloud2.pointcloud2_to_xyz_array(
            self.cloud, remove_nans=True)

        # Apply lidar to car frame transformation
        # Use tf average of all bags instead of calculating new transform
        pc_array_tf = self.transform_pc(pc_array, rpy=(0, 0, 0),
                                        translation_xyz=(-2.36, 0, 1.8375))

        # Filter unwanted points (to reduce point cloud size) with passthrough filter
        pc_array_cut = self.reduce_pc(pc_array_tf, (0, 30), (-2, 2), (-1, 2))

        # Convert numpy array to pcl object
        pc_cut = self.array_to_pcl(pc_array_cut)

        # Downsample point cloud using voxel filter to further decrease size
        pc_small = self.voxel_filter(pc_cut, 0.1)

        # Perform RANSAC until no new planes are being detected
        plane_coor = self.plane_detection(pc_small, 20, 4)
        
        # Transform plane points back to original coordinates
        plane_coor_tf = self.transform_pc(plane_coor, rpy=(0, 0, 0),
                                        translation_xyz=(2.36, 0, -1.8375))
        
        return plane_coor_tf
    
    @staticmethod
    def array_to_pcl(pc_array):
        """Get pcl point cloud from numpy array"""
        pc = pcl.PointCloud()
        pc.from_array(pc_array.astype('float32'))
        return pc

    @staticmethod
    def quat_from_vectors(vec1, vec2):
        """Quaternion that aligns vec1 to vec2"""
        # Normalize vectors
        a, b = unit_vector(vec1), unit_vector(vec2)
        c = np.cross(a, b)
        d = np.dot(a, b)

        # Rotation axis
        axis = c / vector_norm(c)
        # Rotation angle (rad)
        ang = np.arctan2(vector_norm(c), d)

        # Quaternion ([x,y,z,w])
        quat = np.append(axis*np.sin(ang/2), np.cos(ang/2))
        return quat

    @staticmethod
    def transform_pc(pc, rpy=(0, 0, 0), translation_xyz=(1.7, 0, 1.7)):
        """Transformation from Lidar frame to car frame. Rotation in rad and translation in m."""
        # Extract euler angles
        roll, pitch, yaw = rpy
        # Extract translations
        transl_x, transl_y, transl_z = translation_xyz

        # Rotation matrix
        rot = euler_matrix(roll, pitch, yaw, 'sxyz')[:3, :3]
        # Apply rotation
        pc_tf = np.inner(pc, rot)

        # Translation to front of the car
        translation_to_front = [transl_x, transl_y, transl_z]
        # Combine rotation and translation
        pc_tf += translation_to_front
        return pc_tf

    @staticmethod
    def reduce_pc(pc, x_range, y_range, z_range):
        """Removes points outside of box"""
        # Filter array
        pc_cut = pc[
            (pc[:, 0] > x_range[0])
            & (pc[:, 0] < x_range[1])
            & (pc[:, 1] > y_range[0])
            & (pc[:, 1] < y_range[1])
            & (pc[:, 2] > z_range[0])
            & (pc[:, 2] < z_range[1])
        ]
        return pc_cut

    @staticmethod
    def voxel_filter(pc, leaf_size):
        """Downsample point cloud using voxel filter"""
        vgf = pc.make_voxel_grid_filter()
        # Leaf_size is the length of the side of the voxel cube in m
        vgf.set_leaf_size(leaf_size, leaf_size, leaf_size)
        pc_filtered = vgf.filter()
        return pc_filtered

    def plane_detection(self, pc, min_points, max_planes):
        """Detects all planes in point cloud"""
        # Count number of iterations
        counter = 0
        # Standard values for ramp angle and distance if no detection
        ramp_stats = []
        # Detect planes until ramp found or conditions not met anymore
        while pc.size > min_points and counter < max_planes:
            # Detect most dominate plane and get inliers and normal vector
            inliers_idx, coefficients = self.ransac(pc)
            # Normal vector of plane
            n_vec = coefficients[:-1]

            # Split pointcloud in outliers of plane and inliers
            plane = pc.extract(inliers_idx)
            pc = pc.extract(inliers_idx, negative=True)

            # Exit if plane is empty (RANSAC did not find anything)
            if plane.size == 0:
                return ramp_stats

            # Ignore planes parallel to the side or front walls
            if self.is_plane_near_ground(n_vec):
                # Check if ramp conditions are fullfilled
                is_ramp, data = self.ramp_detection(
                    plane, n_vec, (3, 9), (2, 6))
                # Ramp conditions met
                if is_ramp:
                    return data
            counter += 1
        return ramp_stats

    def ramp_detection(self, plane, n_vec, angle_range, width_range):
        """Checks if conditions to be considered a ramp are fullfilled."""
        # Convert pcl plane to numpy array
        plane_array = plane.to_array()

        # Calculate angle [deg] between normal vector of plane and ground
        angle = self.angle_calc([0, 0, 1], n_vec)
        # Get ramp width (difference between y-values)
        width = max(plane_array[:, 1]) - min(plane_array[:, 1])
        # Ramp distance (average x-value of nearest points of the plane)
        n_nearest = 10
        dist = np.mean(np.sort(plane_array[:n_nearest, 0]))

        # Assert ramp angle and width thresholds
        if (angle_range[0] <= angle <= angle_range[1]
                and width_range[0] <= width <= width_range[1]):
            return True, plane_array
        return False, plane_array

    def ransac(self, pc):
        """Finds inliers and normal vector of dominant plane"""
        # 50?
        seg = pc.make_segmenter_normals(50)
        # Doubles the speed if True
        seg.set_optimize_coefficients(True)
        seg.set_model_type(pcl.SACMODEL_NORMAL_PLANE)
        seg.set_method_type(pcl.SAC_RANSAC)
        # How close a point must be to model to be considered inlier
        seg.set_distance_threshold(0.11)
        # normal_distance_weight?
        seg.set_normal_distance_weight(0.01)
        # How many tries
        seg.set_max_iterations(100)
        indices, coefficients = seg.segment()
        return indices, coefficients

    def split_pc(self, pc, inliers):
        """Extract detected plane from point cloud and split into two pcs"""
        # Get point cooridnates of plane
        detected_plane = [pc[i] for i in inliers]
        # Point cloud of detected plane (inliers)
        pc_inliers = pc.extract(inliers)

        # Point cloud of outliers
        outlier_indices = list(
            set(np.arange(pc.size)).symmetric_difference(inliers))
        pc_outliers = pc.extract(outlier_indices)

        return pc_outliers, detected_plane, pc_inliers

    def is_plane_near_ground(self, v, threshold=0.8):
        """Returns True if plane is on the ground (and false if e.g. side wall)"""
        return abs(v[2]) > threshold

    def angle_calc(self, v1, v2, degrees=True):
        """Calculate angle between two vectors (planes)"""
        # Assuming both vectors can be rotated alongside one axis to be aligned
        dot = np.dot(v1, v2)
        if dot <= 1:
            angle = np.arccos(dot)
        else:
            angle = 0

        if degrees is True:
            return np.degrees(angle)
        else:
            return angle


In [ ]:
# Create instance of class (using standard parameters):
vd = VisualDetection()
# Extract detected plane at specifc frame
plane = vd.spin(lidar[frame])

# Project 3D plane points onto 2D plane
plane_img_points, _ = cv2.projectPoints(plane, rvec, tvec, cameraMatrix, distCoeffs)
plane_img_points = plane_img_points.reshape(len(plane_img_points), 2)

fig = px.imshow(img)
fig.add_trace(
    go.Scatter(
        x=df["x"],
        y=df["y"],
        mode="markers",
        marker=dict(size=3, 
                    opacity=0.5, 
                    color=df["dist"], 
                    showscale=True, 
                    colorbar=dict(x=0.85,
                                  y=0.75,
                                  len=0.5,
                                  tickfont=dict(color="white"), 
                                  title=dict(text="Distance [m]", 
                                             font=dict(color="white")))),
    )
)
fig.add_trace(
    go.Scatter(
        x=plane_img_points[:, 0],
        y=plane_img_points[:, 1],
        mode="markers",
        marker=dict(size=4, opacity=0.5, color="light green"),
    )
)
# fig.update_layout(width=img.shape[1], height=img.shape[0], showlegend=False)
fig.update_layout(width=img.shape[1], height=img.shape[0], showlegend=False, 
                  font_family="Serif", font_size=25, 
                  margin_l=5, margin_t=5, margin_b=5, margin_r=5)
# Hide axes and make sure it is not bigger than the image
fig.update_xaxes(range=[0, img.shape[1]], visible=False)
fig.update_yaxes(range=[img.shape[0], 0], visible=False)
fig.show()
fig.write_image("ramp_pdf33.pdf")
import plotly.io as pio
#save a figure of 300dpi, with 1.5 inches, and  height 0.75inches
# pio.write_image(fig, "test.pdf", width=img.shape[1], height=img.shape[0], scale=1)